#### 한국어 코퍼스 전처리
- 데이터셋 : Korpora에서 로딩  
- 형태소분석기 설정  
- 단어사전 생성

In [7]:
# 모듈 로딩
from Korpora import Korpora
from konlpy.tag import *
import spacy

from torchtext.data.utils import get_tokenizer
from torchtext.vocab import *
from torch.utils.data import Dataset, DataLoader

import pandas as pd
import numpy as np


In [8]:
## 데이터 로딩
nsmc=Korpora.load("nsmc")


    Korpora 는 다른 분들이 연구 목적으로 공유해주신 말뭉치들을
    손쉽게 다운로드, 사용할 수 있는 기능만을 제공합니다.

    말뭉치들을 공유해 주신 분들에게 감사드리며, 각 말뭉치 별 설명과 라이센스를 공유 드립니다.
    해당 말뭉치에 대해 자세히 알고 싶으신 분은 아래의 description 을 참고,
    해당 말뭉치를 연구/상용의 목적으로 이용하실 때에는 아래의 라이센스를 참고해 주시기 바랍니다.

    # Description
    Author : e9t@github
    Repository : https://github.com/e9t/nsmc
    References : www.lucypark.kr/docs/2015-pyconkr/#39

    Naver sentiment movie corpus v1.0
    This is a movie review dataset in the Korean language.
    Reviews were scraped from Naver Movies.

    The dataset construction is based on the method noted in
    [Large movie review dataset][^1] from Maas et al., 2011.

    [^1]: http://ai.stanford.edu/~amaas/data/sentiment/

    # License
    CC0 1.0 Universal (CC0 1.0) Public Domain Dedication
    Details in https://creativecommons.org/publicdomain/zero/1.0/

[Korpora] Corpus `nsmc` is already installed at C:\Users\KDP-26\Korpora\nsmc\ratings_train.txt
[Korpora] Corpus `nsmc` is already installed at C:\Users\KD

In [9]:
nsmc.train[0], nsmc.test[0]

(LabeledSentence(text='아 더빙.. 진짜 짜증나네요 목소리', label=0),
 LabeledSentence(text='굳 ㅋ', label=1))

In [10]:
nsmcdf = pd.DataFrame(nsmc.test)
nsmcdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    50000 non-null  object
 1   label   50000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 781.4+ KB


In [11]:
nsmcdf

,text,label
0,굳 ㅋ,1
1,GDNTOPCLASSINTHECLUB,0
2,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0
...,...,...
49995,오랜만에 평점 로긴했네ㅋㅋ 킹왕짱 쌈뽕한 영화를 만났습니다 강렬하게 육쾌함,1
49996,의지 박약들이나 하는거다 탈영은 일단 주인공 김대희 닮았고 이등병 찐따 OOOO,0
49997,그림도 좋고 완성도도 높았지만... 보는 내내 불안하게 만든다,0
49998,절대 봐서는 안 될 영화.. 재미도 없고 기분만 잡치고.. 한 세트장에서 다 해먹네,0


In [14]:
import pandas as pd
from konlpy.tag import Okt
import re
from gensim.models import Word2Vec

# 1. 데이터 로드
nsmcdf = pd.read_csv('nsmcdf.csv')

# 2. NaN 값 및 문자열이 아닌 값 처리
nsmcdf['text'] = nsmcdf['text'].fillna('').astype(str)

# 3. 형태소 분석기 설정
okt = Okt()

# 4. 불용어 리스트 설정 (확장 가능)
stopwords = ['이', '그', '저', '것', '들', '에서', '이다', '있다', '합니다', '했는데', '그리고']

# 5. 특수 문자 및 이모티콘 제거 함수
def clean_text(text):
    text = re.sub(r"[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "", text)  # 한글 및 공백만 남김
    return text

# 6. 형태소 분석 및 어간 추출, 불용어 제거 함수
def preprocess(text):
    text = clean_text(text)  # 특수 문자 및 이모티콘 제거
    tokens = okt.morphs(text, stem=True)  # 어간 추출 및 형태소 분석
    tokens = [word for word in tokens if word not in stopwords]  # 불용어 제거
    return tokens

# 7. 전처리 과정 적용
nsmcdf['tokens'] = nsmcdf['text'].apply(preprocess)

# 8. Word2Vec 임베딩
# Word2Vec 모델 학습을 위해 형태소로 나뉜 리뷰 리스트를 만듭니다.
sentences = nsmcdf['tokens'].tolist()

# Word2Vec 모델 학습
model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

# 9. 단어 벡터 확인 (예: '영화'라는 단어의 벡터 값)
print(model.wv['영화'])

# 10. 임베딩된 벡터를 리뷰 단위로 변환 (리뷰에 포함된 단어 벡터들의 평균을 구함)
def get_embedding(tokens):
    vectors = [model.wv[word] for word in tokens if word in model.wv]
    if vectors:
        return sum(vectors) / len(vectors)
    else:
        return [0] * 100  # 모든 단어가 없을 경우 0 벡터

nsmcdf['embedding'] = nsmcdf['tokens'].apply(get_embedding)

# 11. 전처리된 데이터와 벡터 데이터 저장
nsmcdf.to_csv('processed_nsmcdf.csv', index=False)

# 전처리된 데이터 확인
print(nsmcdf[['text', 'tokens', 'embedding']].head())



[ 1.63445175e-01  7.07372248e-01 -4.42687515e-03  7.29035556e-01
 -2.69910395e-01 -1.54856455e+00  8.19632411e-01  1.94397235e+00
 -4.71358120e-01 -1.36218667e+00  5.12669124e-02 -1.64666748e+00
 -1.71267599e-01  1.18082869e+00 -9.37696934e-01  2.63804674e-01
  5.93839347e-01  1.11893333e-01 -9.16041195e-01 -1.89117515e+00
  7.92187452e-01 -2.42198691e-01  8.45437884e-01 -1.90169406e+00
  8.16037118e-01  5.98661184e-01  6.39135242e-02  5.68732500e-01
 -1.04932773e+00  6.10081613e-01 -3.96209449e-01  5.80888867e-01
  1.60086060e+00 -2.55155873e+00 -1.07419896e+00 -5.50744653e-01
  8.76545429e-01 -2.84380049e-01  1.06280357e-01 -1.87668753e+00
 -1.74153984e+00  3.33074391e-01 -1.07205498e+00  3.32204789e-01
  4.11988460e-02  2.13659764e-03  1.46576867e-01  8.38023007e-01
  4.62503731e-01  6.25509679e-01 -2.37953410e-01 -3.06563318e-01
 -3.36767048e-01  6.89480960e-01  1.06985271e+00 -4.00539011e-01
  3.99440853e-03 -1.04543853e+00 -1.13167286e-01  1.25664580e+00
  1.11216523e-01 -9.74052

In [15]:
nsmcdf

,text,label,tokens,embedding
0,굳 ㅋ,1,"[굳다, ㅋ]","[-0.24233203, 0.5606317, 0.23306501, -0.270053..."
1,GDNTOPCLASSINTHECLUB,0,[],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0,"[뭐, 야, 평점, 은, 나쁘다, 않다, 점, 짜다, 리, 는, 더, 더욱, 아니다]","[-0.29252866, 0.2312499, 0.082995586, 0.051471..."
3,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0,"[지루하다, 않다, 완전, 막장, 임, 돈, 주다, 보기, 에는]","[-0.6132557, 0.4329818, 0.32804203, 0.4547819,..."
4,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0,"[만, 아니다, 별, 다섯, 개, 주다, 왜, 로, 나오다, 제, 심기, 를, 불편...","[-0.37561065, 0.34398335, 0.24822886, 0.002539..."
...,...,...,...,...
49995,오랜만에 평점 로긴했네ㅋㅋ 킹왕짱 쌈뽕한 영화를 만났습니다 강렬하게 육쾌함,1,"[오랜, 만, 에, 평점, 로, 기다, 하다, ㅋㅋ, 킹왕짱, 쌈뽕, 한, 영화, ...","[-0.20398843, 0.34620863, 0.1821121, 0.0289946..."
49996,의지 박약들이나 하는거다 탈영은 일단 주인공 김대희 닮았고 이등병 찐따 OOOO,0,"[의지, 박약, 이나, 하다, 탈영, 은, 일단, 주인공, 김대희, 닮다, 이등병,...","[0.010189229, 0.06543085, 0.3101252, -0.004607..."
49997,그림도 좋고 완성도도 높았지만... 보는 내내 불안하게 만든다,0,"[그림, 도, 좋다, 완성, 도도, 높다, 보다, 내내, 불안하다, 만들다]","[-0.18202488, 0.06247441, 0.36434203, 0.015333..."
49998,절대 봐서는 안 될 영화.. 재미도 없고 기분만 잡치고.. 한 세트장에서 다 해먹네,0,"[절대, 보다, 서다, 안, 되다, 영화, 재미, 도, 없다, 기분, 만, 잡, 치...","[-0.0787021, 0.24936514, 0.37208602, 0.2545075..."


In [1]:
import pandas as pd
import numpy as np
from konlpy.tag import Okt
import re
from gensim.models import Word2Vec

# 1. 데이터 로드
nsmcdf = pd.read_csv('nsmcdf.csv')

# 2. NaN 값 및 문자열이 아닌 값 처리
nsmcdf['text'] = nsmcdf['text'].fillna('').astype(str)

# 3. 형태소 분석기 설정
okt = Okt()

stopwords = [
    '이', '그', '저', '것', '들', '에서', '이다', '있다', '합니다', '했는데', '그리고',  # 기존 불용어
    '영화', '배우', '감독', '연기', '스토리', '작품', '관객', '장면',  # 영화 관련 단어
    '정말', '너무', '매우', '진짜', '거의', '그냥', '그렇게', '정도', '약간', '많이',  # 일반적인 감정 표현
    '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '점', '몇'  # 숫자 및 평점 관련
]

# 5. 특수 문자 및 이모티콘 제거 함수
def clean_text(text):
    text = re.sub(r"[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "", text)  # 한글 및 공백만 남김
    return text

# 6. 형태소 분석 및 어간 추출, 불용어 제거 함수
def preprocess(text):
    text = clean_text(text)  # 특수 문자 및 이모티콘 제거
    tokens = okt.morphs(text, stem=True)  # 어간 추출 및 형태소 분석
    tokens = [word for word in tokens if word not in stopwords]  # 불용어 제거
    return tokens

# 7. 전처리 과정 적용
nsmcdf['tokens'] = nsmcdf['text'].apply(preprocess)

# 8. 패딩을 위해 전체 리뷰에서 가장 긴 리뷰의 길이 계산
max_len = max(nsmcdf['tokens'].apply(len))

# 9. 패딩 함수 구현 (numpy 사용)
def pad_sequences(sequences, max_len, padding_value=0):
    padded_sequences = []
    for seq in sequences:
        if len(seq) < max_len:
            # 부족한 부분을 패딩 값으로 채움
            padded_seq = seq + [padding_value] * (max_len - len(seq))
        else:
            padded_seq = seq[:max_len]  # 최대 길이를 초과하는 경우 자름
        padded_sequences.append(padded_seq)
    return np.array(padded_sequences)

# 10. 패딩 적용
padded_tokens = pad_sequences(nsmcdf['tokens'], max_len=max_len, padding_value=0)

# 11. Word2Vec 임베딩
# Word2Vec 모델 학습을 위해 형태소로 나뉜 리뷰 리스트를 만듭니다.
sentences = nsmcdf['tokens'].tolist()

# Word2Vec 모델 학습
model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

# 12. 임베딩된 벡터를 리뷰 단위로 변환 (리뷰에 포함된 단어 벡터들의 평균을 구함)
def get_embedding(tokens):
    vectors = [model.wv[word] for word in tokens if word in model.wv]
    if vectors:
        return sum(vectors) / len(vectors)
    else:
        return [0] * 100  # 모든 단어가 없을 경우 0 벡터

nsmcdf['embedding'] = nsmcdf['tokens'].apply(get_embedding)

# 13. 패딩된 토큰 및 임베딩된 벡터 데이터 저장
nsmcdf['padded_tokens'] = list(padded_tokens)

nsmcdf.to_csv('processed_nsmcdf_with_padding.csv', index=False)

# 전처리된 데이터 확인
print(nsmcdf[['text', 'tokens', 'padded_tokens', 'embedding']].head())



                                                text  \
0                                                굳 ㅋ   
1                               GDNTOPCLASSINTHECLUB   
2             뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아   
3                   지루하지는 않은데 완전 막장임... 돈주고 보기에는....   
4  3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??   

                                              tokens  \
0                                            [굳다, ㅋ]   
1                                                 []   
2       [뭐, 야, 평점, 은, 나쁘다, 않다, 짜다, 리, 는, 더, 더욱, 아니다]   
3               [지루하다, 않다, 완전, 막장, 임, 돈, 주다, 보기, 에는]   
4  [만, 아니다, 별, 다섯, 개, 주다, 왜, 로, 나오다, 제, 심기, 를, 불편...   

                                       padded_tokens  \
0  [굳다, ㅋ, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...   
1  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   
2  [뭐, 야, 평점, 은, 나쁘다, 않다, 짜다, 리, 는, 더, 더욱, 아니다, 0...   
3  [지루하다, 않다, 완전, 막장, 임, 돈, 주다, 보기, 에는, 0, 0, 0, ...   
4  [만, 아니다, 별, 다섯, 개, 주다, 왜, 로, 나오다, 제, 심기, 를,

In [18]:
nsmcdf

,text,label,tokens,embedding,padded_tokens
0,굳 ㅋ,1,"[굳다, ㅋ]","[-0.33484825, 0.32911414, 0.3305453, -0.384770...","[굳다, ㅋ, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
1,GDNTOPCLASSINTHECLUB,0,[],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0,"[뭐, 야, 평점, 은, 나쁘다, 않다, 점, 짜다, 리, 는, 더, 더욱, 아니다]","[-0.20795915, 0.1596063, 0.22442976, -0.138179...","[뭐, 야, 평점, 은, 나쁘다, 않다, 점, 짜다, 리, 는, 더, 더욱, 아니다..."
3,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0,"[지루하다, 않다, 완전, 막장, 임, 돈, 주다, 보기, 에는]","[-0.53114486, 0.32970157, 0.42415795, 0.189413...","[지루하다, 않다, 완전, 막장, 임, 돈, 주다, 보기, 에는, 0, 0, 0, ..."
4,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0,"[만, 아니다, 별, 다섯, 개, 주다, 왜, 로, 나오다, 제, 심기, 를, 불편...","[-0.3465681, 0.37144765, 0.3665356, 0.04980259...","[만, 아니다, 별, 다섯, 개, 주다, 왜, 로, 나오다, 제, 심기, 를, 불편..."
...,...,...,...,...,...
49995,오랜만에 평점 로긴했네ㅋㅋ 킹왕짱 쌈뽕한 영화를 만났습니다 강렬하게 육쾌함,1,"[오랜, 만, 에, 평점, 로, 기다, 하다, ㅋㅋ, 킹왕짱, 쌈뽕, 한, 영화, ...","[-0.28669906, 0.36891663, 0.27964586, 0.082011...","[오랜, 만, 에, 평점, 로, 기다, 하다, ㅋㅋ, 킹왕짱, 쌈뽕, 한, 영화, ..."
49996,의지 박약들이나 하는거다 탈영은 일단 주인공 김대희 닮았고 이등병 찐따 OOOO,0,"[의지, 박약, 이나, 하다, 탈영, 은, 일단, 주인공, 김대희, 닮다, 이등병,...","[-0.14153339, 0.09592761, 0.37027124, 0.089974...","[의지, 박약, 이나, 하다, 탈영, 은, 일단, 주인공, 김대희, 닮다, 이등병,..."
49997,그림도 좋고 완성도도 높았지만... 보는 내내 불안하게 만든다,0,"[그림, 도, 좋다, 완성, 도도, 높다, 보다, 내내, 불안하다, 만들다]","[-0.073332466, 0.10699739, 0.39355737, -0.0842...","[그림, 도, 좋다, 완성, 도도, 높다, 보다, 내내, 불안하다, 만들다, 0, ..."
49998,절대 봐서는 안 될 영화.. 재미도 없고 기분만 잡치고.. 한 세트장에서 다 해먹네,0,"[절대, 보다, 서다, 안, 되다, 영화, 재미, 도, 없다, 기분, 만, 잡, 치...","[-0.17351946, 0.22059187, 0.3102518, 0.1530826...","[절대, 보다, 서다, 안, 되다, 영화, 재미, 도, 없다, 기분, 만, 잡, 치..."


텍스트 전처리  
트콘화==>최적 토큰화 패키지 사용  
정제( 불용어 , 구두점, 개발자 지정 제거 문자)  
단어사전(정수)  
문장 ==> 수치화  
문장 길이 통일, 패딩  
정수 수치화 ==> One-Hot Encoding 변환 : 파이토치 필요없음  

[모델]  
임베딩층 : 차원 축소  
-RNN/LSTM/GRU 층  